## pba-modul2-id-text-classification

Inisialisasi

In [1]:
import numpy as np
import pandas as pd
import modSpellChecker as sc
from contractions_id import CONTRACTION_MAP
import re
import nltk
import string
from nltk.stem import WordNetLemmatizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import gensim
from gensim.models import Word2Vec
from gensim import models
#from pattern.en import tag
from nltk.corpus import wordnet as wn
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from gensim import corpora, models
import sklearn
#from normalization import normalize_corpus
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from datetime import datetime
import csv

In [2]:
dataset = pd.read_csv('dataartikel.csv')
dataset

,Judul,Artikel,Jenis
0,Rupiah masih rawan bergejolak,Nilai tukar rupiah terhadap dolar Amerika Seri...,ekonomi
1,SRC sasar ritel tradisional jadi pilar ekonomi...,Perkembangan teknologi telah menyebabkan perub...,ekonomi
2,Kiat memaksimalkan Facebook untuk bisnis,Media sosial kini bukan hanya menjadi alat unt...,ekonomi
3,Kiat menambah relasi bisnis,"Dalam menjalankan bisnis, menjalin relasi puny...",ekonomi
4,Prinsip dasar merintis bisnis,Merintis usaha memang bukan pekerjaan mudah. A...,ekonomi
...,...,...,...
1304,Genoa 1 - 1 Sampdoria - Liga Serie A 2018/2019,Genoa dan Sampdoria sama-sama gagal memetik po...,sport
1305,Villarreal 2 - 1 Real Betis - La Liga 2018/2019,Villarreal menjaga langkah untuk mendekatkan d...,sport
1306,"PSM kembali ke puncak klasemen, Persib resmi t...",Persija Jakarta harus rela kembali ke peringka...,sport
1307,Hasil imbang yang pahit bagi AC Milan,"""Hasil imbang yang pahit."" Begitu laman resmi ...",sport


In [3]:
dataset.shape

(1309, 3)

In [4]:
df2 = dataset.groupby(['Jenis'])['Artikel'].count()
df2


Jenis
ekonomi          100
entertainment    134
gaya hidup       200
news             266
sains-tekno      202
sport            407
Name: Artikel, dtype: int64

In [5]:
label = dataset.iloc[:,2]
feature = dataset.iloc[:,1]

In [6]:
feature[0:10]

0    Nilai tukar rupiah terhadap dolar Amerika Seri...
1    Perkembangan teknologi telah menyebabkan perub...
2    Media sosial kini bukan hanya menjadi alat unt...
3    Dalam menjalankan bisnis, menjalin relasi puny...
4    Merintis usaha memang bukan pekerjaan mudah. A...
5    Perdagangan di Bursa Efek Indonesia (BEI) ditu...
6    Indeks Harga Saham Gabungan (IHSG) terkoreksi ...
7    Proses ratifikasi tujuh perjanjian perdagangan...
8    Pemerintah mematangkan skema pembiayaan fasilt...
9    Anak muda zaman sekarang boleh jadi tidak asin...
Name: Artikel, dtype: object

In [7]:
label[0:10]

0    ekonomi
1    ekonomi
2    ekonomi
3    ekonomi
4    ekonomi
5    ekonomi
6    ekonomi
7    ekonomi
8    ekonomi
9    ekonomi
Name: Jenis, dtype: object

In [8]:
character =['z','y','x','w','v','u','t','s','r','q','p','o','n','m','l','k','j','i','h','g','f','e','d'
',','.',';',':','-','...','?','!','(',')','[',']','{','}','<','>','"','/','\'','#','-','@']
def repeatcharNormalize(text):
    for i in range(len(character)):
        charac_long = 5
        while charac_long>=2:
            char=character[i]*charac_long
            text=text.replace(char,character[i])
            charac_long-=1
        return text

In [9]:
def spellNormalize(text):
    spellCheck = []
    for i in text:
        if i not in character:
            j=sc.correction(i)
            spellCheck.append(j)
        else:
            spellCheck.append(i)
        return spellCheck

In [10]:
def tokenize_text(text):
    tokens=nltk.word_tokenize(text)
    tokens=[token.strip() for token in tokens]
    return tokens

In [11]:
def expand_contractions(text, contraction_mapping):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                        flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                               if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())
        expanded_contraction = first_char + expanded_contraction[1:]
        return expanded_contraction
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'","", expanded_text)
    return expanded_text

In [12]:
def stemmer_text(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = stemmer.stem(text)
    return text

In [13]:
def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub('',token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


In [14]:
factory = StopWordRemoverFactory()
stopword_list = factory.get_stop_words()

In [15]:
def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [16]:
def normalize_corpus(corpus, tokenize=False):
    normalized_corpus = []
    for text in corpus:
        text = expand_contractions(text, CONTRACTION_MAP)
        text = stemmer_text(text)
        text = remove_special_characters(text)
        text = repeatcharNormalize(text)
        text = remove_stopwords(text)
        normalized_corpus.append(text)
        if tokenize:
            text = tokenize_text(text)
            text = spellNormalize(text)
            normalized_corpus.append(text)
    # print(normalized_corpus)
    return normalized_corpus

In [17]:
def prepare_datasets(corpus, labels, test_data_proportion=0.3):
    train_X, test_X, train_Y, test_Y = train_test_split(corpus, labels,
                                                        test_size=0.33,
                                                        random_state=42)
    return train_X, test_X, train_Y, test_Y

In [18]:
train_corpus, test_corpus, train_labels, test_labels = prepare_datasets(feature,
                                                                        label,
                                                                        test_data_proportion=0.3)

In [19]:
print(train_corpus)

376     Gonjang-ganjing soal siapa pelatih tim nasiona...
578     Kotak hitam, walau baru sebagian, telah ditemu...
494     Hiu prasejarah megalodon sedang hangat diperbi...
314     Seharusnya, Persib Bandung memiliki peluang be...
1098    Sejak menyutradarai Ngenest (2015) sebagai fil...
                              ...                        
1095    Wiro Sableng: Pendekar Kapak Maut Naga Geni 21...
1130    Pemindai sidik jari saat ini menjadi salah sat...
1294    Napoli dan Chievo sama-sama gagal memetik poin...
860     Banyak orang merasa takut dan gelisah ketika h...
1126    Orang utan adalah salah satu hewan darat palin...
Name: Artikel, Length: 877, dtype: object


In [20]:
print(test_corpus)

1148    Mata merupakan jendela jiwa. Inilah yang diman...
1049    Coldplay akan merilis sebuah dokumenter berjud...
982     Nakita.id - Sarita Abdul Mukti dan Faisal Harr...
808     Akhir-akhir ini, melancong semakin digemari ba...
1195    Musik ternyata tidak hanya berdampak pada manu...
                              ...                        
715     Perkiraan pertumbuhan ekonomi dunia dipangkas....
1246    Pada pengujung 2018, seharusnya masyarakat dap...
814     Media sosial kini bukan hanya menjadi alat unt...
978     Grid.ID - Tak banyak yang tahu tentang sifat a...
885     Masa-masa mencari kerja biasanya penuh dengan ...
Name: Artikel, Length: 432, dtype: object


Start Train

In [21]:
# norm_train_corpus = normalize_corpus(train_corpus)


In [22]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# # Nama file CSV yang ingin Anda buat
# csv_file = '../Modul2/norm_train_corpus_{current_datetime}.csv'

# # Buka file CSV untuk penulisan
# with open(csv_file, 'w', newline='') as file:
#     writer = csv.writer(file)

#     for text in norm_train_corpus:
#         writer.writerow([text])

In [23]:
# norm_train_corpus[0:10]

In [24]:
# norm_test_corpus = normalize_corpus(test_corpus)

In [25]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# # Nama file CSV yang ingin Anda buat
# csv_file = '../Modul2/norm_test_corpus _{current_datetime}.csv'

# # Buka file CSV untuk penulisan
# with open(csv_file, 'w', newline='') as file:
#     writer = csv.writer(file)

#     for text in norm_test_corpus:
#         writer.writerow([text])

In [26]:
# norm_test_corpus[0:10]

Ambil hasil train dan test yang sudah di simpan cvs

train set simpan

In [27]:
train_set = pd.read_csv('train_coprus.csv')
train_set.head(1)

,text
0,"gonjangganjing,soal,siapa,latih,tim,nasional,i..."


In [28]:
array_train = train_set.values
array_train[0:2]

array([['gonjangganjing,soal,siapa,latih,tim,nasional,indonesia,depan,jawab,lalu,rapat,komite,eksekutif,exco,pssi,senin,17,9,2018,malam,federasi,nama,luis,milla,besut,skuat,garuda,putus,sekaligus,jawab,spekulasi,edar,beberapa,hari,akhir,kontrak,milla,tak,panjang,pssi,spekulasi,sebut,maklum,kontrak,akhir,pascaasian,games,milla,kerap,gagal,penuh,target,misal,ajang,sea,games,2017,latih,usia,51,tahun,target,raih,emas,nyata,garuda,u23,perunggu,asi,games,patok,masuk,empat,besar,hansamu,yama,pranata,dkk,finis,16,besar,capai,sebut,tak,masalah,pssi,banyak,suara,minta,milla,tahan,lihat,main,timnas,baik,tak,capai,target,pssi,milla,bagai,latih,timnas,beri,respons,positif,ucap,aggota,exco,pssi,refrizal,goal,indonesia,sama,muka,wakil,ketua,umum,pssi,joko,driyono,rapat,malam,turut,jumlah,poin,pertama,soal,putus,angkat,milla,sang,latih,merespon,baik,tawar,aju,tiga,sambung,joko,bicara,waktu,datang,milla,milla,tengah,libur,jalan,latih,lisensi,uefa,pro,kira,datang,indonesia,9,oktober,hari,laga,uji,coba,l

In [29]:
norm_train_corpus=array_train.flatten().tolist()
norm_train_corpus[10]

'salah,satu,pamer,teknologi,besar,dunia,ifa,internationale,funkausstellung,depan,mata,acara,langsung,tanggal,31,agustus,hingga,5,september,2018,lin,expocenter,city,jerman,ajang,besar,biasa,manufaktur,perangkat,teknologi,samsung,lg,sony,huawei,nokia,hingga,lenovo,bakal,kenal,pamer,bagai,macam,perangkat,gawai,baru,mulai,ponsel,hingga,speaker,pintar,tv,laptop,banyak,ifa,selalu,jadi,sumber,teknologi,konsumen,alat,rumah,tangga,bahkan,saing,consumer,electronic,show,pamer,barang,elektronik,besar,amerika,serikat,selenggara,las,vegas,ikut,manufaktur,nama,gawai,apa,bakal,pamer,lin,bagaimana,rangkum,beberapa,sumber,galaxy,note,9,galaxy,watch,memang,luncur,arti,samsung,habis,stok,pamer,ifa,lansir,the,verge,samsung,bakal,pajang,perangkat,tv,baru,8k,qled,tv,beberapa,tahun,akhir,samsung,memang,manfaat,ifa,luncur,perangkat,lacak,bugar,earbud,nirkabel,usaha,sebut,sempat,pamer,tv,resolusi,8k,ces,2018,hingga,berita,tulis,sebut,berapa,banyak,model,keluar,lin,blackberry,tampak,optimistis,produk,damba,gemar

test train

In [30]:
test_set = pd.read_csv('test.csv')
test_set.head(1)

,text
0,"mata,rupa,jendela,jiwa,manfaat,artificial,inte..."


In [31]:
print(len(test_set))

432


In [32]:
array_test = test_set.values
array_test[0:2]

array([['mata,rupa,jendela,jiwa,manfaat,artificial,inteligence,ai,cerdas,buat,baca,pribadi,turut,buah,teliti,baru,terbit,jurnal,frontiers,in,human,neuroscience,teliti,kembang,buah,teknologi,baru,manfaat,cerdas,buat,deteksi,gera,mata,uak,pribadi,orang,kerja,kata,mampu,revolusi,bagaimana,manusia,komunikasi,mesin,teknologi,depan,sistem,ponsel,pintar,paham,prediksi,perilaku,potensi,tawar,layan,lebih,personal,teknologi,sebut,robot,tugas,tani,orang,lanjut,usia,mobil,kemudi,sendiri,gim,video,interaktif,teliti,university,of,south,australia,kerja,sama,univeristy,of,stuttgart,max,planck,institute,for,informatics,jerman,flinders,university,australia,selatan,rancang,algoritma,mampu,ajar,mesin,kemampuan,komputer,tindak,cara,eksplisit,diprogram,menggunakan,data,kumpul,42,flinders,university,algoritma,ajar,mesin,rekam,gera,mata,laku,tugas,hari,headset,lacak,mata,buat,sensormotoric,instruments,kena,headset,lama,puluh,menit,kemudian,teliti,periksa,cara,silang,temu,kuesioner,biasa,indikasi,ciri,pribadi,

In [33]:
norm_test_corpus=array_test.flatten().tolist()
norm_test_corpus[10]

'satu,juta,botol,plastik,beli,tiap,menit,50,persen,plastik,sekali,pakai,8,juta,ton,plastik,akhir,laut,10,persen,semua,sampah,dunia,plastik,fakta,sebut,rupa,bukti,rusa,alam,manusia,ungkap,situs,world,environment,day,meski,fakta,atas,lihat,cara,global,bukan,arti,negara,bebas,beban,tanggung,jawab,sampah,plastik,bukti,indonesia,rupa,negara,hasil,sampah,nomor,besar,dunia,turut,data,menteri,lingkung,hidup,hutan,klhk,satu,dekade,akhir,sampah,plastik,tingkat,11,persen,jadi,16,persen,tahun,2050,komposisi,sampah,plastik,indonesia,prediksi,capai,40,persen,gera,ubah,perilaku,novrizal,tahar,direktur,kelola,sampah,klhk,kata,lima,bentuk,sampah,plastik,indonesia,kantong,jadi,bentuk,paling,banyak,lanjut,sedot,kemudian,kotak,makan,styrofoam,botol,saset,khusus,kantong,sedot,styrofoam,benar,kata,novrizal,talkshow,no,straw,movement,jakarta,convention,center,jumat,20,7,novrizal,kata,bukan,benda,wajib,dulu,baik,laku,aktivitas,makan,kini,jadi,benda,wajib,makin,banyak,orang,makin,gunung,sampah,hasil,lebih,para

Langkah 4: Ekstraksi Fitur

In [34]:
#Fungsi untuk mengekstraksi feature menggunakan TF-IDF Model
def tfidf_transformer(bow_matrix):
    transformer = TfidfTransformer(norm='l2',
                                    smooth_idf=True,
                                    use_idf=True)
    tfidf_matrix = transformer.fit_transform(bow_matrix)
    return transformer, tfidf_matrix

In [35]:
def tfidf_extractor(corpus, ngram_range=(1,1)):
    vectorizer = TfidfVectorizer(min_df=1,
                                    norm='l2',
                                    smooth_idf=True,
                                    use_idf=True,
                                    ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

In [36]:
tfidf_vectorizer, tfidf_train_features = tfidf_extractor(norm_train_corpus)


In [37]:
tfidf_test_features = tfidf_vectorizer.transform(norm_test_corpus)

Langkah 5: Model Training

In [38]:
from sklearn.linear_model import SGDClassifier
train_features=tfidf_train_features
train_labels=train_labels
test_features=tfidf_test_features
test_labels=test_labels

In [39]:
df_train = pd.DataFrame()
df_train['norm_train'] = norm_train_corpus
df_train['train_labels'] = train_labels
df_test = pd.DataFrame()
df_test['norm_test'] = norm_test_corpus
df_test['test_labels'] = test_labels

In [40]:
df_train.head()

,norm_train,train_labels
0,"gonjangganjing,soal,siapa,latih,tim,nasional,i...",ekonomi
1,"kotak,hitam,baru,temu,kini,semua,utama,keluarg...",ekonomi
2,"hiu,prasejarah,megalodon,sedang,hangat,bincang...",ekonomi
3,"persib,bandung,milik,peluang,besar,juara,liga,...",NaN
4,"sejak,sutradara,ngenest,2015,bagai,film,panjan...",ekonomi


In [41]:
df_test.head()

,norm_test,test_labels
0,"mata,rupa,jendela,jiwa,manfaat,artificial,inte...",NaN
1,"coldplay,rilis,buah,dokumenter,judul,a,head,fu...",NaN
2,"nakita,id,sarita,abdul,mukti,faisal,harris,res...",NaN
3,"akhir,lancong,makin,gari,banyak,kalang,banyak,...",ekonomi
4,"musik,nyata,dampak,manusia,buah,studi,baru,ung...",NaN


In [42]:
train_features[0:1]

<1x23213 sparse matrix of type '<class 'numpy.float64'>'
	with 314 stored elements in Compressed Sparse Row format>

In [43]:
train_labels[0:1]

376    sport
Name: Jenis, dtype: object

In [44]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

In [45]:
clf = SGDClassifier(loss='hinge', max_iter=100)
#clf = LinearSVC(penalty='l2', loss='squared_hinge', max_iter=100)
#clf = MultinomialNB()

In [46]:
print("Jumlah sampel dalam train_features:", train_features.shape[0])
print("Jumlah sampel dalam train_labels:", len(train_labels))
print("Jumlah sampel dalam test_features:", test_features.shape[0])
print("Jumlah sampel dalam test_labels:", len(test_labels))

Jumlah sampel dalam train_features: 877
Jumlah sampel dalam train_labels: 877
Jumlah sampel dalam test_features: 432
Jumlah sampel dalam test_labels: 432


In [47]:
clf.fit(train_features, train_labels)
predictions = clf.predict(test_features)
print("Label test: \n" + format(test_labels[:10]))
print("Prediction test: \n"+ format(predictions[:10]))

Label test: 
1148      sains-tekno
1049    entertainment
982     entertainment
808        gaya hidup
1195      sains-tekno
240             sport
1118      sains-tekno
596              news
924        gaya hidup
65            ekonomi
Name: Jenis, dtype: object
Prediction test: 
['sains-tekno' 'entertainment' 'news' 'gaya hidup' 'sains-tekno' 'sport'
 'sains-tekno' 'news' 'gaya hidup' 'ekonomi']


In [48]:
print("Accuration: "+format(clf.score(test_features,test_labels)))

Accuration: 0.9282407407407407


In [49]:
def get_metrics(true_labels, predicted_labels):
    print('Accuracy: ', np.round(metrics.accuracy_score(true_labels,
                                                predicted_labels),2))
    print('Precision: ', np.round(metrics.precision_score(true_labels,
                                                    predicted_labels,
                                                    average='weighted'),2))
    print('Recall: ', np.round(metrics.recall_score(true_labels,
                                                            predicted_labels,
                                                            average='weighted'),2))
    print('F1 Score: ', np.round(metrics.f1_score(true_labels,
                                                    predicted_labels,
                                                    average='weighted'),2))


In [50]:
get_metrics(true_labels=test_labels,predicted_labels=predictions)

Accuracy:  0.93
Precision:  0.93
Recall:  0.93
F1 Score:  0.93


In [51]:
test = ('Sebuah kapsul NASA mendarat di gurun Utah, AS, pada Minggu (24/9 waktu '
        'setempat. Kapsul tersebut berisi sampel dari asteroid Benny yang dikumpulkan '
        'oleh wahana antariksa OSIRIS-REx milik NASA pada Oktober 2020 lalu. ' +
        'OSIRIS-REx telah menghabiskan waktu selama 7 tahun di luar angkasa untuk '
        'menunggu momen yang tepat mengambil potongan asteroid Bennu. Kini, sampel '
        'tersebut berhasil dibawa OSIRIS-REx ke Bumi, mendarat dengan selamat di '
        'Tempat Uji dan Pelatihan Angkatan Udara AS di Utah pada Minggu (24/4) pukul '
        '08.52 waktu setempat.')


In [52]:
test_eko = ('PT Bank Mandiri (Persero) Tbk (BMRI) melepas seluruh kepemilikan '
            'sahamnya di PT AXA Insurance Indonesia. Meski tak lagi punya saham, Bank '
            'Mandiri tidak akan mengakhiri kerja sama bancassurance dengan AXA Insurance '
            'Indonesia. ' +
            'Secara rinci, sisa 138.000 saham milik Bank Mandiri dilepas ke dua '
            'pemegang saham, yakni Anil Panjwani dan Manoj Ramkrashin Tolani. '
            'Masing-masing mendapat 69.000 saham atau sekitar 10 persen dari modal '
            'ditempatkan dan disetor milik AXA Insurance Indonesia.')


In [53]:
test_or = ('PSSI telah mengumumkan skuad Timnas Indonesia yang akan berlaga '
          'dalam Kualifikasi Piala Dunia 2026 melawan Brunei Darussalam. Dalam daftar '
          '25 pemain tersebut, terdapat satu nama yang absen, yaitu Ivar Jenner, pemain '
          'Jong FC Utrecht. ' +
          'Keberhasilan Timnas Indonesia melawan Brunei Darussalam dalam dua '
          'leg pertandingan ini akan membawa mereka melangkah ke babak kedua '
          'Kualifikasi Piala Dunia 2026. Tim Indonesia akan bersaing di Grup F babak '
          'kedua bersama dengan Irak, Vietnam, dan Filipina.')


In [54]:
test_enter = ('Setelah berkompetisi dalam Busan International Film Festival '
            '(BIFF) 2023, kini film bergenre detektif “24 Jam Bersama Gaspar” merilis '
            'trailer resminya. Trailer tersebut sempat dikeluarkan oleh akun YouTube '
            'resmi Visinema @Visinema Pictures pada Selasa (26/9/2023) kemarin. Di dalam '
            'trailer tersebut, tokoh Gaspar diperankan oleh aktor Reza Rahadian. ' +
            'Melalui sinopsis resmi film 24 Jam Bersama Gaspar, tokoh bernama '
            '“Gaspar” merupakan seorang detektif partikelir yang akan dinyatakan mati '
            'dalam tenggat waktu 24 jam. Dalam kurun waktu itu, Gaspar berupaya keras '
            'untuk membalaskan dendam dan merampok sebuah toko perhiasan yang ia incar. '
            'Keinginannya itu ditimbulkan berkat munculnya petunjuk mengenai hilangnya '
            'Kirana, seorang gadis yang menjadi teman masa kecil Gaspar.')



In [55]:
test_gaya = ('Berbagai modus pinjaman pribadi atau pinpri menjamur di media '
            'sosial. Tak sedikit masyarakat yang lantas memilih pinpri sebagai alternatif '
            'pembiayaan karena prosesnya cepat. Perencana Keuangan Nadia Isnuari Harsya '
            'mengatakan, kebanyakan orang yang terlilit pinjaman pribadi atau pinpri '
            'adalah mereka yang terjebak dengan gaya hidup. ' +
            'Ia menekankan, seseorang yang telah berani mengambil pinjaman '
            'berarti harus mengembalikan pinjaman tersebut. Selain itu, masyarakat '
            'dinilai belum dapat meminjam secara pintar. Beberapa peminjam hanya '
            'memikirkan kesenangan sesaat saja tanpa berpikir soal konsekuensinya.')



In [56]:
test_gh = ('Kalangan Gen Z dan Milenial tumbuh di lingkungan dengan kemajuan '
          'teknologi dalam kehidupan sehari-hari. Hal ini pun turut melahirkan '
          'kebiasaan baru, khususnya dalam berbelanja dan memilih metode pembayaran '
          'tanpa uang tunai atau disebut gaya hidup cashless. ' +
          'Selain itu, kenyamanan, kecepatan, dan keamanan yang ditawarkan pun '
          'menjadi faktor-faktor terbesar yang mendorong metode pembayaran digital. '
          'Apalagi di era smartphone dan internet, kemudahan pembayaran digital '
          'memiliki daya tarik tersendiri, baik itu melalui dompet digital, QR, hingga '
          'kartu kredit contactless.')


In [57]:
line = tfidf_vectorizer.transform([test_gaya])
line.shape

(1, 23213)

In [58]:
line

<1x23213 sparse matrix of type '<class 'numpy.float64'>'
	with 22 stored elements in Compressed Sparse Row format>

In [59]:
test = clf.predict(line)

In [60]:
test

array(['gaya hidup'], dtype='<U13')